Notes: how will the heuristic value interact with the return values from minimax? Need to use + and - inf to stop it from prioritizing heuristic

Create a fake game instance to test alphabeta pruning - just needs to return legal moves

Have disabled random.shuffle in isolation.py!

self.score is not implemented

why is the search exception getting called?

In [2]:
from isolation import Board
from pdb import set_trace
import time
import copy
import random

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
def timeit(method):

    def timed(*args, **kw):
        ts = time.time()*1000
        result = method(*args, **kw)
        te = time.time()*1000

        print('%r (%r, %r) %2.2f' % (method.__name__, args, kw, te-ts))
        return result

    return timed

In [4]:
def time_func(func):
    time_limit_millis = 150
    time_millis = lambda: 1000 * timeit.default_timer()
    
    def timed(*args, **kwargs):
        start = time_millis()
        result = method(*args, **kwargs)
        end = time_millis()
        
        print('%r  %2.2f ms' % (method.__name__, (end - start)))
        return result
    
    return time_func

def show_output(func):
    def wrapped(*args, **kwargs):
        output = func(*args, **kwargs)
        print("The result is:", output)
    return wrapped


In [5]:
class SearchTimeout(Exception):
    """Subclass base exception for code clarity. """
    print('SearchTimeout executed')

def custom_score(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    This should be the best heuristic function for your project submission.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    # TODO: finish this function!
    # Starting with a simple one - difference in the number of legal_moves
    active_moves = len(game.get_legal_moves(game.active_player))
    opponent_moves = len(game.get_legal_moves(game.get_opponent(game.active_player)))
    gs_value = active_moves - opponent_moves
    print('heuristic')
    set_trace()
    return gs_value


def custom_score_2(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    # TODO: finish this function!
    raise NotImplementedError


def custom_score_3(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    # TODO: finish this function!
    raise NotImplementedError

SearchTimeout executed


In [99]:

class IsolationPlayer:
    """Base class for minimax and alphabeta agents -- this class is never
    constructed or tested directly.

    ********************  DO NOT MODIFY THIS CLASS  ********************

    Parameters
    ----------
    search_depth : int (optional)
        A strictly positive integer (i.e., 1, 2, 3,...) for the number of
        layers in the game tree to explore for fixed-depth search. (i.e., a
        depth of one (1) would only explore the immediate sucessors of the
        current state.)

    score_fn : callable (optional)
        A function to use for heuristic evaluation of game states.

    timeout : float (optional)
        Time remaining (in milliseconds) when search is aborted. Should be a
        positive value large enough to allow the function to return before the
        timer expires.
    """
    def __init__(self, search_depth=3, score_fn=custom_score, timeout=10.):
        self.search_depth = search_depth
        self.score = score_fn
        self.time_left = None
        self.TIMER_THRESHOLD = timeout


class MinimaxPlayer(IsolationPlayer):
    """Game-playing agent that chooses a move using depth-limited minimax
    search. You must finish and test this player to make sure it properly uses
    minimax to return a good move before the search time limit expires.
    """

    def get_move(self, game, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        **************  YOU DO NOT NEED TO MODIFY THIS FUNCTION  *************

        For fixed-depth search, this function simply wraps the call to the
        minimax method, but this method provides a common interface for all
        Isolation agents, and you will replace it in the AlphaBetaPlayer with
        iterative deepening search.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left

        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        best_move = (-1, -1)

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
            return self.minimax(game, self.search_depth)

        except SearchTimeout:
            pass  # Handle any actions required after timeout as needed

        # Return the best move from the last completed search iteration
        return best_move
   

    
    def min_value(self, game, depth):
        """ Return the value for a win (+1) if the game is over,
        otherwise return the minimum value over all legal child
        nodes.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            print('min value timed out')
            self.score(game, game.active_player())
            
        if depth <= 0:
            return 0

        val = float('inf')
        for move in game.get_legal_moves():
            val = min(val, self.max_value(game.forecast_move(move), depth-1))
        return val

    def max_value(self, game, depth):
        """ Return the value for a loss (-1) if the game is over,
        otherwise return the maximum value over all legal child
        nodes.
        """
        
        if self.time_left() < self.TIMER_THRESHOLD:
            print('max value timed out')
            self.score(game, game.active_player())
            
        if depth <= 0:
            return 0        
            
        val = float('-inf')
        for move in game.get_legal_moves():
            val = max(val, self.min_value(game.forecast_move(move), depth-1))
        return val


    def minimax(self, game, depth):
        """Implement depth-limited minimax search algorithm as described in
        the lectures.

        This should be a modified version of MINIMAX-DECISION in the AIMA text.
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Minimax-Decision.md

        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.

            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """
        
        
        
        if self.time_left() < self.TIMER_THRESHOLD:
            print('Minimax timed out')
            raise SearchTimeout()

        # TODO: finish this function!
        best_score = float('-inf')
        best_move = (-1,-1)
        
        for move in game.get_legal_moves():
            move_score = self.min_value(game.forecast_move(move), depth - 1)
            print(move, move_score)
            if move_score > best_score:
                best_score = move_score
                best_move = move
        return best_move


class AlphaBetaPlayer(IsolationPlayer):
    """Game-playing agent that chooses a move using iterative deepening minimax
    search with alpha-beta pruning. You must finish and test this player to
    make sure it returns a good move before the search time limit expires.
    """

    def get_move(self, game, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        Modify the get_move() method from the MinimaxPlayer class to implement
        iterative deepening search instead of fixed-depth search.

        **********************************************************************
        NOTE: If time_left() < 0 when this function returns, the agent will
              forfeit the game due to timeout. You must return _before_ the
              timer reaches 0.
        **********************************************************************

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left

        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        best_move = (-1, -1)

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
            return self.alphabeta(game, self.search_depth)

        except SearchTimeout:
            pass
            # CH: what to do here?

        # Return the best move from the last completed search iteration
        return best_move

    def min_value(self, game, depth, alpha, beta):
        if self.time_left() < self.TIMER_THRESHOLD:
            print('min value timed out')
            self.score(game, game.active_player())

        if depth <= 0:
            return 0, alpha
        
        val = float('inf')
        set_trace()
        for move in game.get_legal_moves():
            v, b = self.max_value(game.forecast_move(move), depth - 1, alpha, beta)
            if v <= alpha: 
                break
            else:
                val = min(val, v)
                beta = min(beta, b)
            
        if val > alpha: 
            alpha = val
        return val, alpha
    
    def max_value(self, game, depth, alpha, beta):
        if self.time_left() < self.TIMER_THRESHOLD:
            print('min value timed out')
            self.score(game, game.active_player())

        if depth <= 0:
            return 0, beta
        
        val = float('-inf')
        set_trace()
        for move in game.get_legal_moves():
            v, a = self.min_value(game.forecast_move(move), depth - 1, alpha, beta)
            if v >= beta: 
                break
            else:
                val = max(val, v)
                alpha = max(alpha, a)
            
        if val < beta: 
            beta = val
        return val, beta

        
    def alphabeta(self, game, depth, alpha=float("-inf"), beta=float("inf")):
        """Implement depth-limited minimax search with alpha-beta pruning as
        described in the lectures.

        This should be a modified version of ALPHA-BETA-SEARCH in the AIMA text
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Alpha-Beta-Search.md

        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        alpha : float
            Alpha limits the lower bound of search on minimizing layers

        beta : float
            Beta limits the upper bound of search on maximizing layers

        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.

            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()  
            
        best_score = float('-inf')
        best_move = (-1,-1)
        
        self.alpha = alpha
        self.beta = beta
        
        for move in game.get_legal_moves():
            move_score, _ = self.min_value(game.forecast_move(move), depth - 1, self.alpha, self.beta)
            print(move, move_score)
            if move_score > best_score:
                best_score = move_score
                best_move = move
        return best_move


In [100]:
time_limit = 10**200

In [ ]:
one = AlphaBetaPlayer(50, custom_score)
two = copy.deepcopy(one)
small_board = Board(one, two, 1,2)
small_board.play(time_limit)
print(small_board.to_string())

> <ipython-input-99-fd1fa6a2fb69>(243)min_value()
-> for move in game.get_legal_moves():
(Pdb) l
238  	        if depth <= 0:
239  	            return 0, alpha
240  	
241  	        val = float('inf')
242  	        set_trace()
243  ->	        for move in game.get_legal_moves():
244  	            v, b = self.max_value(game.forecast_move(move), depth - 1, alpha, beta)
245  	            if v <= alpha:
246  	                break
247  	            else:
248  	                val = min(val, v)
(Pdb) move
*** NameError: name 'move' is not defined
(Pdb) print(game.to_string())
     0
0  | 1 | 
1  |   | 

(Pdb) game.get_legal_moves()
[(1, 0)]
(Pdb) n
> <ipython-input-99-fd1fa6a2fb69>(244)min_value()
-> v, b = self.max_value(game.forecast_move(move), depth - 1, alpha, beta)
(Pdb) self.max_value(game.forecast_move(move), depth - 1, alpha, beta)
(-inf, -inf)
(Pdb) n
--Call--
> c:\users\chris\google drive\python\ai\isolation\aind-isolation\isolation\isolation.py(102)forecast_move()
-> def forecast_

In [96]:
one = MinimaxPlayer(50, custom_score)
two = copy.deepcopy(one)
small_board = Board(one, two, 3,5)
small_board.play(time_limit)
print(small_board.to_string())

(0, 0) -inf
(1, 0) -inf
(2, 0) -inf
(3, 0) -inf
(4, 0) -inf
(0, 1) -inf
(1, 1) -inf
(2, 1) -inf
(3, 1) -inf
(4, 1) -inf
(0, 2) -inf
(1, 2) -inf
(2, 2) -inf
(3, 2) -inf
(4, 2) -inf


(<__main__.MinimaxPlayer at 0x21b99709ba8>, [], 'forfeit')

     0   1   2
0  |   |   |   | 
1  |   |   |   | 
2  |   |   |   | 
3  |   |   |   | 
4  |   |   |   | 



In [61]:
#@timeit
def minimax_test():
    one = MinimaxPlayer(3, custom_score)
    two = copy.deepcopy(one)
    small_board = Board(one, two, 4,5)
    result = small_board.play(time_limit)
    print(small_board.to_string())
    return result
    
#@timeit
def alphabeta_test():
    one = AlphaBetaPlayer()
    two = AlphaBetaPlayer()
    small_board = Board(one, two, 4,5)
    small_board.play(time_limit)
    print(small_board.to_string())

In [62]:
minimax_test()

     0   1   2   3
0  | - | - | - |   | 
1  | - | 2 | - | - | 
2  | - | - | - |   | 
3  | 1 |   | - |   | 
4  |   |   |   |   | 



(<__main__.MinimaxPlayer at 0x21b996b94a8>,
 [[0, 0],
  [1, 0],
  [1, 2],
  [0, 2],
  [2, 0],
  [2, 1],
  [0, 1],
  [1, 3],
  [2, 2],
  [3, 2],
  [3, 0],
  [1, 1]],
 'forfeit')

In [63]:
alphabeta_test()

     0   1   2   3
0  | - | - | - | 1 | 
1  | - | 2 | - | - | 
2  | - | - | - |   | 
3  |   |   | - |   | 
4  |   |   |   |   | 



In [77]:
print(small_board.to_string())

     0   1   2   3
0  | - | - | - |   | 
1  | - |   | - | - | 
2  | - | - | 1 |   | 
3  |   |   | 2 |   | 



In [171]:
small_board.apply_move(one.get_move(small_board,test_tl))
print(small_board.to_string())

'minimax' ((<__main__.MinimaxPlayer object at 0x000001F5D5C67FD0>, <isolation.isolation.Board object at 0x000001F5D5C67EF0>, 3), {}) 1.00
     0   1   2
0  | 1 |   |   | 
1  |   |   |   | 



In [172]:
small_board.apply_move(two.get_move(small_board,test_tl))
print(small_board.to_string())

'minimax' ((<__main__.MinimaxPlayer object at 0x000001F5D5C67B00>, <isolation.isolation.Board object at 0x000001F5D5C67EF0>, 3), {}) 0.00
     0   1   2
0  | 1 |   |   | 
1  |   |   | 2 | 



In [168]:
print(small_board.to_string())

     0   1   2
0  | 1 |   |   | 
1  |   |   | 2 | 

